# DoorDash Application - Data Analysis Projects

## Overview

In this set of data analysis projects, we conducted various analyses related to the DoorDash ecosystem, focusing on the DashMart Master Data. The analyses cover areas such as inventory management, product sales, and order fulfillment times.

## Project

### Master Data Enhancement

**Objective:** Conduct comprehensive data analysis on DoorDash DashMart Master Data.
- **Tasks:**
  - Evaluated slow-moving products in inventory by calculating the average days a product stays in inventory.
  - Identified slow-moving products based on the average days criterion.
  - Determined top-selling products by analyzing sales data.
  - Analyzed order fulfillment times for delivered orders.

## Job Capabilities Addressed

### 1. Master Data Management

- **Achievement:** Developed processes to identify slow-moving products in inventory, enhancing data quality and supporting strategic decisions.

### 2. Data-Driven Decision Making

- **Achievement:** Utilized SQL and data analysis techniques to extract insights on top-selling products and order fulfillment times, enabling informed decision-making.

### 3. Collaboration and Communication

- **Achievement:** Communicated findings through clear and concise visualizations, fostering collaboration with cross-functional teams (Product, Engineering, and Operations).

### 4. Problem Solving

- **Achievement:** Applied analytical skills to solve complex problems, demonstrating a proactive and innovative approach.

## Conclusion

This project, contained in a single notebook, showcases my data analysis skills, highlighting the ability to derive actionable insights from data and contribute to the optimization of DoorDash DashMart operations.

In [26]:
import pandas as pd
import sqlite3


In [15]:
import sqlite3


# Connect

In [17]:
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create a sample inventory table


In [19]:
cursor.execute('''
    CREATE TABLE inventory (
        product_id INT,
        product_name VARCHAR(50),
        quantity INT
    )
''')


# Fetch all rows from the cursor


In [20]:
cursor.execute('SELECT * FROM inventory')
rows = cursor.fetchall()


# Display the result


In [21]:
rows

[]

# Create a DataFrame from the result


In [27]:
df = pd.DataFrame(rows, columns=['product_id', 'product_name', 'quantity'])


# Display the DataFrame


In [28]:
df

,product_id,product_name,quantity


# Insert sample data


In [29]:
cursor.executemany('''
    INSERT INTO inventory (product_id, product_name, quantity)
    VALUES (?, ?, ?)
''', [(101, 'Widget A', 15), (102, 'Widget B', 8), (103, 'Gadget X', 25)])


# Commit changes


In [30]:
conn.commit()


# View the sample inventory


In [31]:
cursor.execute('SELECT * FROM inventory')
rows = cursor.fetchall()
df = pd.DataFrame(rows, columns=['product_id', 'product_name', 'quantity'])
df

,product_id,product_name,quantity
0,101,Widget A,15
1,102,Widget B,8
2,103,Gadget X,25


# Run the query to identify products with low inventory


In [33]:
cursor.execute('SELECT product_id, product_name, quantity FROM inventory WHERE quantity < 10')
rows_low_inventory = cursor.fetchall()
df_low_inventory = pd.DataFrame(rows_low_inventory, columns=['product_id', 'product_name', 'quantity'])
df_low_inventory

,product_id,product_name,quantity
0,102,Widget B,8


# Calculate the total value of the inventory


In [35]:
cursor.execute('SELECT SUM(quantity) as total_quantity, SUM(quantity) as total_value FROM inventory')
inventory_summary = cursor.fetchone()
total_quantity, total_value = inventory_summary

print(f'Total Quantity: {total_quantity}')
print(f'Total Value: ${total_value}')

Total Quantity: 48
Total Value: $48


# Create a sample sales table


In [37]:
cursor.execute('''
    CREATE TABLE sales (
        order_id INT,
        product_id INT,
        quantity_sold INT
    )
''')



# Insert sample sales data



In [38]:
cursor.executemany('''
    INSERT INTO sales (order_id, product_id, quantity_sold)
    VALUES (?, ?, ?)
''', [(1, 101, 5), (2, 102, 8), (3, 103, 12)])



# Commit changes


In [39]:
conn.commit()



# View the sample sales data


In [40]:
cursor.execute('SELECT * FROM sales')
rows_sales = cursor.fetchall()
df_sales = pd.DataFrame(rows_sales, columns=['order_id', 'product_id', 'quantity_sold'])
df_sales

,order_id,product_id,quantity_sold
0,1,101,5
1,2,102,8
2,3,103,12


# Identify the top-selling products


In [42]:
cursor.execute('''
    SELECT p.product_id, p.product_name, COALESCE(SUM(s.quantity_sold), 0) as total_sold
    FROM inventory p
    LEFT JOIN sales s ON p.product_id = s.product_id
    GROUP BY p.product_id, p.product_name
    ORDER BY total_sold DESC
    LIMIT 5
''')
top_selling_products = cursor.fetchall()
df_top_selling = pd.DataFrame(top_selling_products, columns=['product_id', 'product_name', 'total_sold'])
df_top_selling

,product_id,product_name,total_sold
0,103,Gadget X,12
1,102,Widget B,8
2,101,Widget A,5


# Insert sample orders data


In [53]:
cursor.executemany('''
    INSERT INTO orders (order_id, order_time, delivery_time, status)
    VALUES (?, ?, ?, ?)
''', [(1, '2023-01-01 12:00:00', '2023-01-01 13:00:00', 'delivered'),
      (2, '2023-01-02 10:00:00', '2023-01-02 11:30:00', 'delivered'),
      (3, '2023-01-03 15:00:00', '2023-01-03 16:30:00', 'in progress')])



# Commit changes


In [54]:
conn.commit()



# View the sample orders data


In [56]:
cursor.execute('SELECT * FROM orders')
rows_orders = cursor.fetchall()
df_orders = pd.DataFrame(rows_orders, columns=['order_id', 'order_time', 'delivery_time', 'status'])
df_orders

,order_id,order_time,delivery_time,status
0,1,2023-01-01 12:00:00,2023-01-01 13:00:00,delivered
1,2,2023-01-02 10:00:00,2023-01-02 11:30:00,delivered
2,3,2023-01-03 15:00:00,2023-01-03 16:30:00,in progress
3,1,2023-01-01 12:00:00,2023-01-01 13:00:00,delivered
4,2,2023-01-02 10:00:00,2023-01-02 11:30:00,delivered
5,3,2023-01-03 15:00:00,2023-01-03 16:30:00,in progress


# Analyze order fulfillment times


In [57]:
cursor.execute('''
    SELECT order_id, strftime('%s', delivery_time) - strftime('%s', order_time) as fulfillment_time
    FROM orders
    WHERE status = 'delivered'
''')
order_fulfillment_times = cursor.fetchall()
df_fulfillment_times = pd.DataFrame(order_fulfillment_times, columns=['order_id', 'fulfillment_time'])
df_fulfillment_times

,order_id,fulfillment_time
0,1,3600
1,2,5400
2,1,3600
3,2,5400


# Display the schema of the inventory table

In [64]:

cursor.execute("PRAGMA table_info('inventory')")
schema_inventory = cursor.fetchall()
print("Schema of the inventory table:")
for column in schema_inventory:
    print(column)


Schema of the inventory table:
(0, 'product_id', 'INT', 0, None, 0)
(1, 'product_name', 'VARCHAR(50)', 0, None, 0)
(2, 'quantity', 'INT', 0, None, 0)


# Display a line break


In [65]:
print("\n" + "="*30 + "\n")


# Display the schema of the orders table


In [66]:
cursor.execute("PRAGMA table_info('orders')")
schema_orders = cursor.fetchall()
print("Schema of the orders table:")
for column in schema_orders:
    print(column)

Schema of the orders table:
(0, 'order_id', 'INT', 0, None, 0)
(1, 'order_time', 'DATETIME', 0, None, 0)
(2, 'delivery_time', 'DATETIME', 0, None, 0)
(3, 'status', 'VARCHAR(50)', 0, None, 0)


# Identify slow-moving products with corrected days_in_inventory calculation


In [70]:
cursor.execute('''
    SELECT product_id, product_name, AVG(julianday('now') - julianday(current_date)) as avg_days_in_inventory
    FROM inventory
    GROUP BY product_id, product_name
    HAVING AVG(julianday('now') - julianday(current_date)) > 30
''')
slow_moving_products = cursor.fetchall()
df_slow_moving_products = pd.DataFrame(slow_moving_products, columns=['product_id', 'product_name', 'avg_days_in_inventory'])
df_slow_moving_products

,product_id,product_name,avg_days_in_inventory


# Analyze order fulfillment times


In [71]:
cursor.execute('''
    SELECT order_id, julianday(delivery_time) - julianday(order_time) as fulfillment_time
    FROM orders
    WHERE status = 'delivered'
''')
order_fulfillment_times = cursor.fetchall()
df_fulfillment_times = pd.DataFrame(order_fulfillment_times, columns=['order_id', 'fulfillment_time'])
df_fulfillment_times

,order_id,fulfillment_time
0,1,0.041667
1,2,0.062500
2,1,0.041667
3,2,0.062500


# Analyze top-selling products


In [72]:
cursor.execute('''
    SELECT p.product_id, p.product_name, SUM(s.quantity_sold) as total_sold
    FROM inventory p
    LEFT JOIN sales s ON p.product_id = s.product_id
    GROUP BY p.product_id, p.product_name
    ORDER BY total_sold DESC
    LIMIT 5
''')
top_selling_products = cursor.fetchall()
df_top_selling_products = pd.DataFrame(top_selling_products, columns=['product_id', 'product_name', 'total_sold'])
df_top_selling_products


,product_id,product_name,total_sold
0,103,Gadget X,12
1,102,Widget B,8
2,101,Widget A,5


# Analyze average order fulfillment times


In [73]:
cursor.execute('''
    SELECT AVG(julianday(delivery_time) - julianday(order_time)) as avg_fulfillment_time
    FROM orders
    WHERE status = 'delivered'
''')
avg_fulfillment_time = cursor.fetchone()[0]
print(f"Average Order Fulfillment Time: {avg_fulfillment_time} days")

Average Order Fulfillment Time: 0.05208333325572312 days
